In [ ]:
import pandas as pd

df = pd.read_csv("../Data/imdb_top_1000.csv")

# Ger rid of useless columns

df.drop(columns=["Poster_Link", "Certificate", "Runtime", "Overview", "Meta_score"], inplace=True)
df.dropna(inplace=True)

# Cleanup data
df.drop([966, 1], inplace=True)
df["Genre"] = df["Genre"].str.split(',')
df["Gross"] = df["Gross"].str.replace(',', '')
df["Gross"] = df["Gross"].astype(int)
df.head()

In [ ]:
# Oldest Movie

title = df.loc[df[["Released_Year"]].idxmin()]['Series_Title'].values[0]
print(f'The oldest movie is "{title}".')

In [ ]:
# Newest Movie

newest_year = df[["Released_Year"]].max()
titles = df.loc[df["Released_Year"].astype(int) == int(newest_year.iloc[0])]['Series_Title'].values.tolist()

print(f"The most recent release year is {int(newest_year.iloc[0])}")
print(f"{len(titles)} entries were released in {int(newest_year.iloc[0])}. \nThey are:")
for t in titles:
    print(t)


In [ ]:
# Top 10 movies by IMDB rating

df.sort_values('IMDB_Rating', ascending=False, inplace=True)
df.head(10)

In [ ]:
# Top movie for each genre
e_df = df.explode("Genre")
e_df = e_df[["Genre", "Series_Title", "No_of_Votes", "Gross"]]
# setup new df of genres
genres = df["Genre"]
# flatten into list
genres = [elem for elems in genres for elem in elems]
# strip out spaces
genres = [genre.strip(' ') for genre in genres]
# convert to dict then back to list to remove duplicates
genres = list(dict.fromkeys(genres))
top_by_genre = []


for genre in genres:
    new_df = e_df[e_df["Genre"].str.contains(genre)]
    top_movie = new_df.loc[new_df[["No_of_Votes"]].idxmax()]['Series_Title'].values[0]
    top_by_genre.append([genre, top_movie])

for movie in top_by_genre:
    print(f"The top movie in {movie[0]} is: {movie[1]}.")

In [ ]:
# Director with the most movies
directors = df["Director"].to_list()
directors = list(dict.fromkeys(directors))
num_rows = 0
most_movies = 0
most_director = ''

for direct in directors:
    new_df = df[df["Director"].str.contains(direct)]
    num_rows = len(new_df.index)
    if num_rows >= most_movies:
        most_movies = num_rows
        most_director = direct
print(f"{most_director} has directed the most movies, a total of {most_movies}.")

In [ ]:
# Star with the most movies

star_df = df[["Series_Title", "Star1", "Star2", "Star3", "Star4"]]
star_df["Stars"] = star_df[["Star1", "Star2", "Star3", "Star4"]].values.tolist()
star_df.drop(["Star1", "Star2", "Star3", "Star4"], inplace=True, axis=1)
star_df = star_df.explode("Stars")
stars = star_df["Stars"]
stars = list(dict.fromkeys(stars))
num_rows = 0
most_movies = 0
most_star = ''

for star in stars:
    new_df = star_df[star_df["Stars"].str.contains(star)]
    num_rows = len(new_df.index)
    if num_rows >= most_movies:
        most_movies = num_rows
        most_star = star
print(f"{most_star} has starred in the most movies, a total of {most_movies}.")

In [ ]:
# Highest grossing movie for each genre
highest_by_genre = []
top_movie = ''

for genre in genres:
    new_df = e_df[e_df["Genre"].str.contains(genre)]
    top_movie = new_df.loc[new_df[["Gross"]].idxmax()]['Series_Title'].values[0]
    highest_by_genre.append([genre, top_movie])

for movie in highest_by_genre:
    print(f"The highest grossing movie in {movie[0]} is: {movie[1]}.")


In [ ]:
# Lowest grossing movie for each director

lowest_by_genre = []
top_movie = ''

for genre in genres:
    new_df = e_df[e_df["Genre"].str.contains(genre)]
    top_movie = new_df.loc[new_df[["Gross"]].idxmin()]['Series_Title'].values[0]
    lowest_by_genre.append([genre, top_movie])

for movie in lowest_by_genre:
    print(f"The lowest grossing movie in {movie[0]} is: {movie[1]}.")

In [ ]:
# Save the dataframe as a parquet file

df.to_parquet('../Data/dataframetoparquet.parquet.gzip', compression='gzip')



In [ ]:
# Aggregate the gross revenue for all 1000 movies.
df.agg({"Gross":['sum', 'mean', 'median', 'min', 'max']})